### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Fall 2018 ###

# Automotive Inspection with the Laser Radar #

#### Kartik Sreenivasan (email address), Matthew Crepea (email address), Talha Sultan (email address), and Eric Brandt (elbrandt@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The [Nikon Laser Radar](https://www.nikonmetrology.com/en-us/product/laser-radar-mv331-351) is a high accuracy, large volume, non-contact, 3D measurement device that is gaining considerable adoption in the automotive industry for [in-line car body inspection](https://www.youtube.com/watch?v=odIYMRyCFDo). This inspection has traditionally been done with a tactile CMM which is a slow process requiring one-of-every-N cars to be spurred off the main assembly line into a carefully controlled 'measurement room'. Automotive manufacturers [can achieve considerable cost and time savings](http://www.nikonmetrology.com/en-gb/product/laser-radar-shop-floor-cmm) by switching to a robot-mounted laser-based measurement system such as the Laser Radar to inspect *all* part instances in-line, without special 1-of-N measurement followed by generalizing results to the remainder of the unmeasured instances. The laser-based measurement technology is mature, however, this relatively new adoption by the short cycle-time automotive industry has placed new pressure on maximizing the quantity and quality of feature measurements that can be made in short, fixed time windows common in assembly line manufacturing. By applying optimization techniques to the measurement plan and process, we can to increase both the quality and quantity of measurements that can be made in a fixed time window.

Automotive engineers routinely specify geometric features that must be inspected on a car when it is in the [Body-In-White](https://en.wikipedia.org/wiki/Body_in_white) phase of manufacturing. Inspection involves measuring the 3D locations and dimensions of 200-300 holes, slots, studs, surfaces, etc relative to fixed locations on the car. The measured location of these features are compared to the nominal locations specified in the CAD drawings. [Statistical process control](https://www.itl.nist.gov/div898/handbook/pmc/section1/pmc12.htm) calculations are performed to evaluate the overal accuracy, precision, reliability, and repeatability of the manufacturing process. The task of scanning each feature and extracting the location of the measured geometry relative to a global ‘car coordinate system’ is well handled by existing software.
![Nikon Laser Radar](LR_Combo.jpg)

However, the process of deciding *how* to measure these features using the Laser Radar is a decision largely made by manufacturing engineers using 'rule-of-thumb' techniques based on their experience with robot and tactile CMM programming. Often these engineers have little experience with laser based measurement systems. The opportunity to optimize this scenario lies in several sub-problems:

* *Determine the optimal order in which to measure a list of features.* Pointing the measuring laser of the device to each feature location requires moving a mirror in azimuth and elevation, both of which have known velocity and acceleration profiles and limits. The optimization task is: For a given fixed laser radar location relative to the car, find an ordering of a given list of features to be measured that minimizes total measurement time.

* *Determine the optimal location at which to locate the Laser Radar.* Each feature type that is measured (hole, slot, stud, etc) has certain line-of-sight geometry characteristics that affect the confidence in the accuracy of the final measurement. For example, stud features achieve best accuracy when the laser’s line of sight has a particular angle relative to the stud’s normal direction. Holes, on the other hand, are most accurate when the laser’s line of sight is closer to the normal direction of the hole. The optimization task is: Given the line-of-sight geometry characteristics for each type of feature geometry, find an optimal placement of the Laser Radar device such that the accuracy of measurement of all n features (of varying types) in a given list is maximized. 
 

## 2. Mathematical model ##

### 2.1 Coordinate Systems
Within the work cell, there are many coordinate systems being used. To keep the focus of this work on the optimization component, we will simplify the work cell to two fundamental coordinate systems: The Part Coordinate System (PCS), and Machine Coordinate System (MCS). It is important to understand each and be able to freely convert points in space between these coordinate systems.

* **Part coordinate system (PCS)**: This is a 3D Cartesian coordinate system that is the same coordinate system the engineer used in CAD to design the car. In our models, this coordinate system has its origin on the centerline of the car, both side-to-side and front-to-back, and is located underneath the car. The Z-axis is points toward the front of the car, and the Y-axis points upward. Being a right-handed coordinate system, this leaves the X-axis to points toward the driver's side door.  All features to be measured are defined in this coordinate system. Also, the location and orientation of the Laser Radar in the work cell is specified in this coordinate system.
![Part Coordinate System](PCS.png)

* **Machine coordinate system (MCS)**: This is the coordinate system from the viewpoint of the Laser Radar. (The historical name 'Machine' comes from the world of CNC and tactile CMM devices). The Laser Radar measures in a [spherical coordinate system](https://en.wikipedia.org/wiki/Spherical_coordinate_system): It measures *range* using reflected signal from an IR laser, and the laser itself can be 'steered' in *azimuth* and *elevation* by means of a motorized mirror that spins on one axis and pivots on an orthoganal axis. The *azimuth*, or $\phi$, ranges from -180 to 180 degrees $(-\pi, \pi)$, with 0 being the front of the Laser Radar. The *elevation*, or ranges from -45 to +45 degrees relative to horizontal. To stay with standard ISO spherical coordinate conventions, we will define this as $\theta$, to be measured as the inclination angle, or the angle relative to vertical, giving it a valid range of $(\frac{\pi}{4}, \frac{3\pi}{4})$, with $\frac{\pi}{2}$ being horizontal. See diagram below (courtesy Wikipedia). The $\phi$ and $\theta$ angles of the mirror are recorded by two highly accurate encoders inside the Laser Radar, and when synchronized with the laser's range measurement, form a spherical coordinate triple $(r, \theta, \phi)$ that originates from the center of the mirror on the laser radar.
![Spherical Coordinate System](SphericalCS.png)

The first transformation required for optimzation is converting Spherical MCS to Cartesian MCS. This is accomplished using standard trigonometry:
\begin{align*}
x &= r \sin \theta \cos \phi \\
y &= r \sin \theta \sin \phi \\
z &= r \cos \theta
\end{align*}
And likewise, we can convert Cartesian MCS to Spherical MCS:
\begin{align*}
r &= \sqrt{x^2 + y^2 + z^2} \\
\theta &= \arccos\frac{z}{\sqrt{x^2 + y^2 + z^2}} = \arctan \frac{\sqrt{x^2+y^2}}{z}\\
\phi &= \arctan \frac{y}{x} \\
\end{align*}

Note the manipulation of $\theta$ by trig identities to reformulate using the $\arctan$ function. For correct results in quadrants 2 and 4, it is necessary to use the function `atan2()` programmatically to account for sign changes.

The second transformation required for optimization is converting Cartesian MCS to PCS. This is accomplished via a standard 3D affine transformation. We define the location and orientation of the Laser Radar within the part coordinate system according to a homogeneous transformation matrix. If the Laser Radar's orientation is described by its 3 orthogonal axis $X$, $Y$, and $Z$ as orthonormal vectors $U$, $V$, and $W$ in PCS, and the Laser Radar's mirror center is described in PCS as point $(LR_x, LR_y, LR_z)$, then the matrix that transforms a point in Cartesian MCS point to the same point in PCS is 
\begin{equation*}
T_{\text{MCS to PCS}} = \begin{bmatrix} 
U_x & V_x & W_x & LR_x \\
U_y & V_y & W_y & LR_y \\
U_z & V_z & W_z & LR_z \\
0   & 0   & 0   & 1
\end{bmatrix}
\end{equation*}
And, of course, the matrix that transforms PCS points to Cartesian MCS points is simply the inverse:
\begin{equation*}
T_{\text{PCS to MCS}} = T_{\text{MCS to PCS}}^{-1} 
\end{equation*}

### 2.2 Measurement Feature Definitions

TODO - (anyone) Explain more formally:

Basically a CSV file with each line defining one feature to be measured:

FeatureType,X,Y,Z,I,J,K

* FeatureType = Hole|Slot|Stud|Rectangle|SurfaceVector|Sphere|.... (we can add others as necessary to make part 2 more interesting)
* X,Z,Y - CAD location in PCS
* I,J,K - CAD Normal direction in PCS

* First feature file now exists: 50_Holes_FrontLeft.csv

We'll need a 2nd file that contains only the 16 entries of the transformation matrix that locates the LR in the PCS.

### 2.3 Laser Radar Motion

TODO anyone (explain more formally):

Speed of travel for azimuth and elevation changes of the mirror: 
Easy: simply define max velocity, e.g. $\frac{\pi}{4}$ radians/sec. Assume accel/decel is instantaneous
Slightly harder: Assume constant accel up to max velocity, then constant decel to finish.

### 2.4 Optimal Measurement Plan Path

### 2.4.1 Formulating as TSP problem

**Inputs:** 
* _LR Location/orientation matrix_: The location of the laser radar with respet to the features to be measured is critical in determining the cost matrix because the nodes to visit are defined in spherical coordinates relative to the mirror of the laser radar.
* _Mirror Azimuth and Elevation velocities_: These velocities determine how fast the laser radar can move its measurement laser from one location to another. For this model, we assume instantaneous acceleration to start and stop the mirror, and constant velocity during travel. This is reasonable for this model because the mirror assembly has relatively low mass and low moment of inertia, so acelerations are fast.
* _Unordered site location list:_ (csv file containing points to visit) This defines the location of the features to be measured in spherical coordinates with respect to the Laser Radar's mirror. The feature definition file described above contains feature locations in cartesian coordinates relative to the car's coordinate system. Before constructing the cost matrix, these cartesian coordinates must be converted to spherical coordinates relative to he Laser Radar's coordinate system. This is transformation is described above.

**Output:** **TODO**
* order in which to measure the features
* time it takes to measure all features.

**Cost Matrix:** **TODO**
* describe

**Optimization Algorithm**:

Once the inputs are properly defined, and a cost matrix is constructed, TSP algorithm can be applied. In our implementation, we implemented four different TSP algorithms:
1. MTZ formulation (from class lecture)
2. Iterative Subtour Elimination (from class lecture)
3. Subtour Elmination using Lazy Constraints (based on 2, but using additional JuMP/Gurobi capability of `@lazyconstraint`)
4. Symmetric Matrix & Lazy Subtour Elimination (based on Gurobi website [example](http://examples.gurobi.com/traveling-salesman-problem/))

We evaluated the performance of each, and settled on \#4 - the Symmetrix Matrix with Lazy Subtour Elimination algorithm. Therefore, only this algorithm will be discussed here, but a thorough comparison of all four can be seen at the accompanying notebook [**HERE (CLICK ME!)**](TSP_Comparison.ipynb). The high-level results of this evaluation are also discussed in section [#4.-Results-and-discussion](#4.-Results-and-discussion).

** TODO: EXPLAIN ALGORITHM HERE.**

### 2.4.1 Optimizing 'Path' versus 'Tour'

Because we are only interested in quickly measuring each feature (visiting each site), we have no interest in returning to the first site at the conclusion of the measurement. The next job for the Laser Radar is not likely to be the same sequence of measurements, so we do not care where the laser is pointing at the conclusion of the measurement. Therefore, we desire a 'path' rather than a 'tour'.

To modify the TSP algorithm to produce a 'path' rather than a 'tour', we made the following changes:
1. Add an additional 'dummy point' to the list of points to visit.
2. Append a row and column to the cost matrix so that the cost to travel from any real point to the 'dummy point' is 0. 
3. This, node with zero cost everywhere gives a 'free pass' for the laser to move from any one single feature to any one single other feature with zero cost (by traveling through the 'dummy point'). The optimization algorithm will naturally choose to take this zero-cost path instead of the most expensive path in its solution under consideration.
4. Solve as usual, and produce the best order to visit the points.
5. Left-shift (with wrap-around) the output order until the dummy point is at the head of the list. The point *after* the dummy point will be the starting point of the path, and the point *before* the dummy point will be the finishing point.
6. Remove the dummy point that is now at the head of the list. This is the optimal 'path' order.


## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
using JuMP, Clp

m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


## 4. Results and discussion ##

### 4.1 TSP Algorithm Evaluation

**TODO: EXPLAIN**
![TSP Comparison](TSP_Comparison_Chart.png)


### 4.2 Scan Path Results
**TODO: Explain**
![Unordered features](50Holes_FrontLeft_No_Opt.png)
![Ordered features](50Holes_FrontLeft_Opt.png)

```
Feature path without optimization: 11.792777 sec
Feature path with optimization:    1.561622 sec
Time savings:                      10.231154 sec
Optimal feature ordering:
[29, 14, 35, 39, 24, 41, 20, 12, 21, 49, 47, 18, 23, 36, 28, 32, 45, 37, 1, 10, 46, 33, 44, 25, 27, 15, 5, 34, 50, 48, 3, 19, 40, 13, 4, 43, 7, 22, 8, 17, 38, 6, 2, 31, 30, 26, 9, 42, 16, 11]
Computational time to optimize:    0.188292 sec
```

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.